# Pixiv Craw

> - ### import repos

In [10]:
import requests
import json
from tqdm import tqdm
from pixiv_crawler.downloader.downloader import Downloader
from pixiv_crawler.config import NETWORK_CONFIG, MODE_CONFIG, USER_CONFIG

> - ### 按照artist的id来craw

1.1 按照画师的id进行爬取（后续会添加函数 按照排名，标签进行爬取）

In [11]:
def fetch_artist_works(artist_id, category):
    # pixiv 的请求返回一般有两种，一种是漫画，一种是插画，还有其他的我懒得搞了，毕竟用的人很少（笑）
    if category not in ["illusts", "manga"]:
        raise ValueError("Invalid category. Use 'illustrations' or 'manga'.")

    url = f"https://www.pixiv.net/ajax/user/{artist_id}/profile/all?lang=zh"

    additional_headers = {
        "x-user-id": "100485309",
        "COOKIE": "first_visit_datetime_pc=2023-11-14%2021%3A14%3A16; p_ab_id=2; p_ab_id_2=5; p_ab_d_id=71834117; __utmz=235335808.1699964057.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _fbp=fb.1.1699964058282.1477207184; yuid_b=ExEzmCA; _gid=GA1.2.850018427.1699964127; _im_vid=01HF6W467E25MGXNXPJHEWPTDQ; device_token=dccafc93aaa8692c65f392c9a7ab00d2; privacy_policy_agreement=6; _ga_MZ1NL4PHH0=GS1.1.1699964577.1.0.1699964580.0.0.0; privacy_policy_notification=0; a_type=0; b_type=1; QSI_S_ZN_5hF4My7Ad6VNNAi=v:0:0; __cf_bm=EcdCq6u9Cawg5WGif5GMjKPfagY8Cn8LOorJQIztgy4-1699977997-0-ASiN6tdcrdSeVoHG+HLRzr4ie44p7eGBLLwf9926P50W+qVTe85kjNknXeTyrx0vQD5R1IFLtLWkOdggMgKe8eCw18QCPpazQ8PGIcyD5i1g; __utma=235335808.537133348.1699964057.1699964057.1699977999.2; __utmc=235335808; __utmt=1; cf_clearance=IS1KlIfrIWG0DyX6j9cU2VDiL2fgV5dfE7a4e3qhBwg-1699977998-0-1-91f80494.de9dec2c.c6f95b74-0.2.1699977998; PHPSESSID=100485309_FWeAxy5e9SMySeu4tvDPCSnMaDbMzocj; c_type=29; __utmv=235335808.|2=login%20ever=no=1^3=plan=premium=1^5=gender=male=1^6=user_id=100485309=1^9=p_ab_id=2=1^10=p_ab_id_2=5=1^11=lang=zh=1; __utmb=235335808.5.10.1699977999; _ga_75BBYNYN9J=GS1.1.1699977999.3.1.1699978046.0.0.0; _ga=GA1.2.362165901.1699964058"
    }

    if category == "illusts":
        additional_headers["Referer"] = f"https://www.pixiv.net/users/{artist_id}/illustrations"
    elif category == "manga":
        additional_headers["Referer"] = f"https://www.pixiv.net/users/{artist_id}/manga"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36",
    }

    headers.update(additional_headers)

    proxies = {
        "https": "127.0.0.1:7890"
    }

    response = requests.get(url, headers=headers, proxies=proxies, timeout=4)

    print(response.json())

    if response.status_code == 200:
        id_group = set(response.json()["body"][category].keys())
        return id_group

In [5]:
artist_id = 3094189
categorys = ["manga", "illusts"]
id = 0

id_group = set()

for category in categorys:
    id_group.update(fetch_artist_works(artist_id=artist_id, category=category))
    print(id_group)
    
artist_artworks_ids_path = "E:\\vscode_clustter\\code\\github\\Animate-Image-Bed\\utils\\crawler\\test\\jsons\\"

file_name = f"{artist_id}-artworksID-collect.json"

artist_dict = {artist_id: list(id_group)}

# 将字典保存到 JSON 文件中
with open(artist_artworks_ids_path + file_name, "w", encoding="utf-8") as json_file:
    json.dump(artist_dict, json_file, ensure_ascii=False, indent=4)



{'error': False, 'message': '', 'body': {'illusts': {'81225309': None, '78834571': None, '73521542': None, '70052155': None, '70024717': None, '69998496': None, '66355934': None, '64089360': None, '60474023': None, '58420472': None, '56471514': None, '54724406': None, '53859115': None, '51455457': None, '47749401': None, '44982128': None, '31957654': None}, 'manga': {'103686781': None, '100082808': None, '89532917': None, '81225213': None, '81225091': None, '78336518': None, '76053184': None, '72066662': None, '69998409': None, '66355845': None, '64089303': None, '60575412': None, '57988139': None, '53969876': None, '51907682': None, '44982929': None, '40466977': None, '37435957': None, '32202507': None}, 'novels': [], 'mangaSeries': [], 'novelSeries': [], 'pickup': [{'id': '103686781', 'title': 'C101新刊「性教育シリーズ総集編2」', 'illustType': 1, 'xRestrict': 1, 'restrict': 0, 'sl': 6, 'url': 'https://i.pximg.net/c/288x288_80_a2/img-master/img/2022/12/17/23/29/50/103686781_p0_square1200.jpg', 'des

1.2 按照tag进行爬取（后续会添加函数 按照排名，标签进行爬取）

In [23]:
def fetch_tag_works(tag_name, category, order):
    # pixiv 的请求返回一般有两种，一种是漫画，一种是插画，还有其他的我懒得搞了，毕竟用的人很少（笑）
    if category not in ["illust", "manga"]:
        raise ValueError("Invalid category. Use 'illustrations' or 'manga'.")

    """url = f"https://www.pixiv.net/ajax/search/illustrations/{tag_name}?"+ \ 
            "&asd" + \
            "lang=zh"""
    
    if category == "illust":
        category_rel = "illustrations"
    else:
        category_rel = "manga"
    
    url = "https://www.pixiv.net/ajax/search/{}/".format(category_rel) + \
            "{}?word={}".format(tag_name, tag_name) + \
            "&order={}".format("popular_d") + \
            f"&mode=all" + "&p=2&s_mode=s_tag&type=illust_and_ugoira&lang=zh"

    """
    /ajax/search/manga/BlueArchive?word=BlueArchive&order=date_d&mode=all&p=1&s_mode=s_tag_full&type=manga&lang=zh&version=6d99637327b03bc728989e9dfd2b67555a8dcbad HTTP/1.1
    """

    additional_headers = {
        "x-user-id": "100485309",
        "COOKIE": "first_visit_datetime_pc=2023-11-14%2021%3A14%3A16; p_ab_id=2; p_ab_id_2=5; p_ab_d_id=71834117; __utmz=235335808.1699964057.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _fbp=fb.1.1699964058282.1477207184; yuid_b=ExEzmCA; _gid=GA1.2.850018427.1699964127; _im_vid=01HF6W467E25MGXNXPJHEWPTDQ; device_token=dccafc93aaa8692c65f392c9a7ab00d2; privacy_policy_agreement=6; _ga_MZ1NL4PHH0=GS1.1.1699964577.1.0.1699964580.0.0.0; privacy_policy_notification=0; a_type=0; b_type=1; QSI_S_ZN_5hF4My7Ad6VNNAi=v:0:0; __cf_bm=EcdCq6u9Cawg5WGif5GMjKPfagY8Cn8LOorJQIztgy4-1699977997-0-ASiN6tdcrdSeVoHG+HLRzr4ie44p7eGBLLwf9926P50W+qVTe85kjNknXeTyrx0vQD5R1IFLtLWkOdggMgKe8eCw18QCPpazQ8PGIcyD5i1g; __utma=235335808.537133348.1699964057.1699964057.1699977999.2; __utmc=235335808; __utmt=1; cf_clearance=IS1KlIfrIWG0DyX6j9cU2VDiL2fgV5dfE7a4e3qhBwg-1699977998-0-1-91f80494.de9dec2c.c6f95b74-0.2.1699977998; PHPSESSID=100485309_FWeAxy5e9SMySeu4tvDPCSnMaDbMzocj; c_type=29; __utmv=235335808.|2=login%20ever=no=1^3=plan=premium=1^5=gender=male=1^6=user_id=100485309=1^9=p_ab_id=2=1^10=p_ab_id_2=5=1^11=lang=zh=1; __utmb=235335808.5.10.1699977999; _ga_75BBYNYN9J=GS1.1.1699977999.3.1.1699978046.0.0.0; _ga=GA1.2.362165901.1699964058"
    }

    if category == "illusts":
        additional_headers["Referer"] = f"https://www.pixiv.net/tags/{tag_name}/illustrations?order=popular_d&s_mode=s_tag"
    elif category == "manga":
        additional_headers["Referer"] = f"https://www.pixiv.net/tags/{tag_name}/manga?order=popular_d&s_mode=s_tag"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36",
    }

    headers.update(additional_headers)

    proxies = {
        "https": "127.0.0.1:7890"
    }

    response = requests.get(url, headers=headers, proxies=proxies, timeout=4)

    print(response.json())

    json_content = response.json()

    # 指定保存路径和文件名
    save_path = "E://vscode_clustter//code//github//Animate-Image-Bed//utils//crawler//test//jsons//re.json"

    # 将 JSON 内容保存到文件
    with open(save_path, "w", encoding="utf-8") as json_file:
        json.dump(json_content, json_file, ensure_ascii=False, indent=4)

    if response.status_code == 200:
        response_data = response.json()["body"][category]["data"]
        id_group = set(entry["id"] for entry in response_data)
        return id_group

In [24]:
from urllib.parse import quote

# ...

tag_name = "早瀬ユウカ"
category = "illust"

id_group = set()

# Encode the tag_name before using it in the URL
encoded_tag_name = quote(tag_name, safe='')

id_group.update(fetch_tag_works(tag_name=encoded_tag_name, category=category, order=True))

print(len(id_group))


{'error': False, 'body': {'illust': {'data': [{'id': '112217482', 'title': 'ユウかと指ハート作り❤️', 'illustType': 0, 'xRestrict': 0, 'restrict': 0, 'sl': 4, 'url': 'https://i.pximg.net/c/250x250_80_a2/custom-thumb/img/2023/10/02/19/03/26/112217482_p0_custom1200.jpg', 'description': '', 'tags': ['ブルーアーカイブ', 'ブルアカ', 'BlueArchive', '早瀬ユウカ', 'ユウカ', '女の子', '太もも', '二人でハート', 'ブルマ', '先生(ブルーアーカイブ)'], 'userId': '87164619', 'userName': 'JJJSSS / ジェース', 'width': 3756, 'height': 6000, 'pageCount': 1, 'isBookmarkable': True, 'bookmarkData': None, 'alt': '#ブルーアーカイブ ユウかと指ハート作り❤️ - JJJSSS / ジェース的插画', 'titleCaptionTranslation': {'workTitle': None, 'workCaption': None}, 'createDate': '2023-10-02T19:03:26+09:00', 'updateDate': '2023-10-02T19:03:26+09:00', 'isUnlisted': False, 'isMasked': False, 'aiType': 1, 'profileImageUrl': 'https://i.pximg.net/user-profile/img/2022/10/18/00/35/43/23478491_456e65e41c59f5689031cf4f2941e510_50.jpg'}, {'id': '101110875', 'title': '競泳水着ユウカ♥', 'illustType': 0, 'xRestrict': 0, 'restri

2.对爬取的图片id进行对应url的提取和集成

In [25]:
print("===== collector start =====")

downloader = Downloader(capacity=1024)

with tqdm(total=len(id_group), desc="collecting urls") as pbar:
    urls = [f"https://www.pixiv.net/ajax/illust/{illust_id}/pages?lang=zh"
            for illust_id in id_group]
    additional_headers = [
        {
            "Referer": f"https://www.pixiv.net/artworks/{illust_id}",
            "x-user-id": "100485309",
            "COOKIE": "first_visit_datetime_pc=2023-11-14%2021%3A14%3A16; p_ab_id=2; p_ab_id_2=5; p_ab_d_id=71834117; __utmz=235335808.1699964057.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _fbp=fb.1.1699964058282.1477207184; yuid_b=ExEzmCA; _gid=GA1.2.850018427.1699964127; _im_vid=01HF6W467E25MGXNXPJHEWPTDQ; device_token=dccafc93aaa8692c65f392c9a7ab00d2; privacy_policy_agreement=6; _ga_MZ1NL4PHH0=GS1.1.1699964577.1.0.1699964580.0.0.0; privacy_policy_notification=0; a_type=0; b_type=1; QSI_S_ZN_5hF4My7Ad6VNNAi=v:0:0; __cf_bm=EcdCq6u9Cawg5WGif5GMjKPfagY8Cn8LOorJQIztgy4-1699977997-0-ASiN6tdcrdSeVoHG+HLRzr4ie44p7eGBLLwf9926P50W+qVTe85kjNknXeTyrx0vQD5R1IFLtLWkOdggMgKe8eCw18QCPpazQ8PGIcyD5i1g; __utma=235335808.537133348.1699964057.1699964057.1699977999.2; __utmc=235335808; __utmt=1; cf_clearance=IS1KlIfrIWG0DyX6j9cU2VDiL2fgV5dfE7a4e3qhBwg-1699977998-0-1-91f80494.de9dec2c.c6f95b74-0.2.1699977998; PHPSESSID=100485309_FWeAxy5e9SMySeu4tvDPCSnMaDbMzocj; c_type=29; __utmv=235335808.|2=login%20ever=no=1^3=plan=premium=1^5=gender=male=1^6=user_id=100485309=1^9=p_ab_id=2=1^10=p_ab_id_2=5=1^11=lang=zh=1; __utmb=235335808.5.10.1699977999; _ga_75BBYNYN9J=GS1.1.1699977999.3.1.1699978046.0.0.0; _ga=GA1.2.362165901.1699964058"
        }
        for illust_id in id_group]

    for url, add_headers in zip(urls, additional_headers):

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36",
        }

        headers.update(add_headers)

        proxies = {
            "https": "127.0.0.1:7890"
        }

        response = requests.get(
                        url, headers=headers,
                        proxies=proxies,
                        timeout=4)

        # 获取 JSON 内容
        json_content = response.json()

        # 指定保存路径和文件名
        save_path = "E://vscode_clustter//code//github//Animate-Image-Bed//utils//crawler//test//jsons//re.json"

        # 将 JSON 内容保存到文件
        with open(save_path, "w", encoding="utf-8") as json_file:
            json.dump(json_content, json_file, ensure_ascii=False, indent=4)

        if response.status_code == 200:
            print(f"url: {url} , resp_code==200, collected Over !" )
            # 因为一个作品可能有多个img ，所以要进行归一化
            collected_urls = set()
            for url in response.json()["body"]:
                collected_urls.add(url["urls"]["original"])
        
        if collected_urls is not None:
            downloader.add(collected_urls)
        pbar.update()

===== collector start =====


collecting urls:   2%|▏         | 1/60 [00:01<01:22,  1.40s/it]

url: https://www.pixiv.net/ajax/illust/109184152/pages?lang=zh , resp_code==200, collected Over !


collecting urls:   3%|▎         | 2/60 [00:01<00:42,  1.37it/s]

url: https://www.pixiv.net/ajax/illust/107840093/pages?lang=zh , resp_code==200, collected Over !


...

url: https://www.pixiv.net/ajax/illust/92434726/pages?lang=zh , resp_code==200, collected Over !


collecting urls:  98%|█████████▊| 59/60 [00:25<00:00,  2.75it/s]

url: https://www.pixiv.net/ajax/illust/110819356/pages?lang=zh , resp_code==200, collected Over !


collecting urls: 100%|██████████| 60/60 [00:25<00:00,  2.34it/s]

url: https://www.pixiv.net/ajax/illust/106223999/pages?lang=zh , resp_code==200, collected Over !


> - ### Download Images

In [26]:
print("===== download start =====")
downloader.download()

===== download start =====
[INFO]: ===== downloader start =====


downloading / flow 396.28MB: 100%|██████████| 166/166 [01:21<00:00,  2.03it/s]

[INFO]: ===== downloader complete =====


396.2766284942627